This notebook shows how to make candidate spacepoints.

We use an extremely simple algorithm to generate possible points.
It simply generates 3D points from the intersection of wires that produce signals at the same time.

This notebook walks through the classes involved in the process of making spacepoints and their labels and then voxelize them.

For running on large datasets, however, use the scripts in `larflow/larmatchnet/larmatch/dataprep/`. Please refer to that folder's README.md for how to run the scripts and where to find the source simulation files.

In [ ]:
import os,sys
import plotly as pl
import plotly.graph_objects as go
import numpy as np
%load_ext autoreload
%autoreload 2

In [ ]:
# on trex, for some reason, need to load ROOT in separate cell before loading icdl modules
import ROOT as rt
from ROOT import std

In [ ]:
from larlite import larlite
from larlite import larutil
from larcv import larcv
from ublarcvapp import ublarcvapp
from larflow import larflow
import lardly

# Load the data file
Below we specify the location of the file we want to use.

What we want is a simulation file with
* simulated data from the detector (the three wire plane images)
* truth information we can use to label generated spacepoints

We want this truth information for ALL trajectories in the images, so we need file with simulated neutrino interactions and cosmic ray particles.

In MicroBooNE, we refer to these files as the "corsika" samples, which derive their name from the Corsika cosmic ray simulation package we use in MicroBooNE. (NB: we know that Corsika over-predicts the number of muons that travel through our detector by a little bit.)

For a list of example files, refer to this [list](https://github.com/NuTufts/icdl/wiki/Locations-of-Tutorial-Files).

In [ ]:
# Specify location of the file we want to open

# For list of tutorial files check out []
inputfile = "/tutorial_files/dlmerged_larflowtruth_mcc9_v13_bnbnue_corsika_run00001_subrun00001.root"
#inputfile = "/home/twongjirad/working/data/mcc9_v13_bnbnue_corsika/dlmerged_larflowtruth_mcc9_v13_bnbnue_corsika_run00001_subrun00001.root"

# we need the name of the TTree that stores our images
WIRE_ADC = "wire"

In [ ]:
# Use ROOT to dump the data of the file
# you'll see a lot of TTree objects. 
# These are essentially tables of data with a specific schema, 
#   with each row of data consisting of one "event".

# For this tutorial we'll need the following TTree(s)
# * 

rfile = rt.TFile(inputfile,"open")
rfile.ls()

We load up our data IO interfaces for both larlite- and larcv-type data.

In [ ]:
TICKBACKWARD = True

ioll = larlite.storage_manager( larlite.storage_manager.kREAD )
ioll.add_in_filename(  inputfile )
ioll.open()

if TICKBACKWARD:
    iolcv = larcv.IOManager( larcv.IOManager.kREAD, "larcv", larcv.IOManager.kTickBackward )
else:
    iolcv = larcv.IOManager( larcv.IOManager.kREAD, "larcv", larcv.IOManager.kTickForward )
iolcv.add_in_file( inputfile )
iolcv.reverse_all_products()
iolcv.initialize()

NENTRIES = iolcv.get_n_entries()
print("Number of entries in file: ",NENTRIES)

# Set the Detector Geometry

To calculate wire intersection points, we need the geometry of the detector.

To speed up spacepoint proposal generation, we actually do not calculate wire-intersections at runtime.
Instead we pre-calculate which wires intersect with which and store these in detector-specific "overlap matrix" files.

These overlap files are generated by code in `icdl/spacepointgen`.

However, DON'T remake these. You can ask someone for these files.

In [ ]:
DETECTOR = "uboone"

# choices: "icarus", "uboone", "sbnd"
overlap_folder = "/tutorial_files/"

if DETECTOR == "icarus":
    detid = larlite.geo.kICARUS
    overlap_matrix_file = overlap_folder+"/output_icarus_wireoverlap_matrices.root"
elif DETECTOR == "uboone":
    detid = larlite.geo.kMicroBooNE
    overlap_matrix_file = overlap_folder+"/output_microboone_wireoverlap_matrices.root"
elif DETECTOR == "sbnd":
    detid = larlite.geo.kSBND
    overlap_matrix_file = "does_not_exist_yet.root"
    
if not os.path.exists(overlap_matrix_file):
    print("WARNING: Could not find overlap matrices")
else:
    print("Found overlap matrix file")
    
larutil.LArUtilConfig.SetDetector(detid)

# Load the Algorithm Classes


In [ ]:
# We are going to define a ROOT file for the algorithms to write out to

try:
    outfile.Close()
except:
    pass

outfile = rt.TFile("out_tutorial_make_truthlabeled_spacepoints.root","recreate")
triptree = rt.TTree("larmatchtriplet","LArMatch triplets") # a new tree to store triplet data

# triplet proposal maker      
tripletmaker = larflow.prep.PrepMatchTriplets()
tripletmaker.set_wireoverlap_filepath( overlap_matrix_file  )
ev_tripdata = std.vector("larflow::prep::MatchTriplets")() # container to store triplet maker output
triptree.Branch("triplet_v",ev_tripdata) # add the container to the triplet tree, triptree

# Keypoint maker
kpana = larflow.keypoints.PrepKeypointData()
#kpana.set_verbosity( larcv.msg.kDEBUG )
kpana.setADCimageTreeName( WIRE_ADC )
outfile.cd()
kpana.defineAnaTree()

# ssnet label data: provides particle label for each spacepoint                                                                                                                                                                          
ssnet = larflow.prep.PrepSSNetTriplet()
outfile.cd()
ssnet.defineAnaTree()

# We make spacepoints from 2D information, so there are some mistakes we fix using various methods
truthfixer = larflow.prep.TripletTruthFixer()


## Run on an Event

In [ ]:
# Now we load an event

ENTRY_NUM = 0
ioll.go_to(ENTRY_NUM)
iolcv.read_entry(ENTRY_NUM)


In [ ]:
# Run the algorithms


# mcpg = ublarcvapp.mctools.MCPixelPGraph()
# mcpg.buildgraphonly( ioll )
# mcpg.printGraph(0,False)
# sys.stdout.flush()
outfile.cd()

# make triplet proposals: function valid for simulation or real data                                                                                                                                                                  
tripletmaker.process( iolcv, WIRE_ADC, WIRE_ADC, 10.0, True )
    
# make good/bad triplet ground truth                                                                                                                                                      
tripletmaker.process_truth_labels( iolcv, ioll, WIRE_ADC )

# fix up some labels
truthfixer.calc_reassignments( tripletmaker, iolcv, ioll )

# # make keypoint score ground truth                                                                                                                                                        
kpana.process( iolcv, ioll )
kpana.make_proposal_labels( tripletmaker )
kpana.fillAnaTree()

# # make ssnet ground truth                                                                                                                                                                 
ssnet.make_ssnet_labels( iolcv, ioll, tripletmaker )

# print("triplet data in output container: ",tripletmaker._match_triplet_v.size())
for imatch in range(tripletmaker._match_triplet_v.size()):
    ev_tripdata.push_back( tripletmaker._match_triplet_v.at(imatch) )
triptree.Fill()

outfile.cd()
# write the data to file and then close the file
kpana.writeAnaTree()
ssnet.writeAnaTree()
triptree.Write()
outfile.Close()

## Visualize the output

In [ ]:
# We generate the TPC boundary lines for plotting and define useful defaults for our figures

import lardly
from lardly.detectoroutline import get_tpc_boundary_plot

tpclines = get_tpc_boundary_plot()

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

plot_layout = {
    "title": "",
    "height":800,
    "margin": {"t": 0, "b": 0, "l": 0, "r": 0},
    "font": {"size": 12, "color": "black"},
    "showlegend": False,
    "plot_bgcolor": "white",
    "paper_bgcolor": "white",
    "scene": {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": 3, "y": 2, "z": 2},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    },
}

# PARTICLE LABEL COLORS
# from larcv/core/DataFormat/DataFormatTypes.h
#     kROIUnknown=0, ///< LArbys
#     kROICosmic,    ///< Cosmics
#     kROIBNB,       ///< BNB
#     kROIEminus,    ///< Electron
#     kROIGamma,     ///< Gamma
#     kROIPizero,    ///< Pi0
#     kROIMuminus,   ///< Muon
#     kROIKminus,    ///< Kaon
#     kROIPiminus,   ///< Charged Pion
#     kROIProton,    ///< Proton
#     kROITypeMax    ///< enum element counter
ssnetcolor = {0:np.array((0,0,0)),     # kROIUnknown                                                                                                                                                   
              1:np.array((255,0,0)),   # kROICosmic (not used)                                                                                                                                       
              2:np.array((0,255,0)),   # kROIBNB (not used)                                                                                                                             
              3:np.array((0,0,255)),   # kROIEminus (e-/e+)                                                                                                                                              
              4:np.array((255,0,255)), # kROIGamma                                                                                                                                                 
              5:np.array((0,255,255)), # kROIPizero                                                                                                                                            
              6:np.array((255,255,0)), # kROImuminus (mu-/mu+)
              7:np.array((123,300,10)),# kROIKminus (k+/k-)
              8:np.array((204,204,255)), # kROIPiminus (pi+/pi-)
              9:np.array((255, 165, 0))} # kProton

kpcolors = {0:np.array((255,0,0)), # nu
            1:np.array((0,255,0)), # track-start
            2:np.array((0,0,255)), # track-end
            3:np.array((255,255,0)), # shower
            4:np.array((0,255,255)), # michel
            5:np.array((255,0,255))} # delt

## Load the spacepoint data file we made

In [ ]:
# Our plotting tools -- and the deep learning frameworks work with numpy arrays.
# We have written C++ functions that output the spacepoints and labels we've generated into
#  such numpy arrays
from ctypes import c_int
from larflow import larflow
larflow.keypoints.LoaderKeypointData

f_v = std.vector("std::string")()
f_v.push_back( "out_tutorial_make_truthlabeled_spacepoints.root" )
kploader = larflow.keypoints.LoaderKeypointData( f_v )
kploader.set_verbosity( larcv.msg.kDEBUG )
kploader.exclude_false_triplets( False )

kploader.load_entry(0)

# the original image data in sparse2D array form
tripletmaker = kploader.triplet_v.at(0).setShuffleWhenSampling( False )

# 2d images                                                                                                                                                                               
wireimg_dict = {}
for p in range(3):
    wireimg = kploader.triplet_v.at(0).make_sparse_image( p )
    wireimg_coord = wireimg[:,:2].astype(np.long)
    wireimg_feat  = wireimg[:,2]
    wireimg_dict["wireimg_coord%d"%(p)] = wireimg_coord
    wireimg_dict["wireimg_feat%d"%(p)] = wireimg_feat


tripdata = kploader.triplet_v.at(0).get_all_triplet_data(True)
spandata = kploader.triplet_v.at(0).get_matchspan_array().astype(np.float32)
spacepoints = kploader.triplet_v.at(0).make_spacepoint_charge_array()

nfilled = c_int(0)
ntriplets = tripdata.shape[0]

kplabel_sigma = 5.0
TPCID = 0
CRYOID = 0
data = kploader.sample_data( ntriplets, nfilled, True, kplabel_sigma, TPCID, CRYOID )
sys.stdout.flush()

#data.update(kpdata)
data.update(spacepoints)
data["truespan_t"] = spandata

print("KeypointLoader returned dictionary with event data. KEYS: ")
print(data.keys())

In [ ]:
# Plot Spacepoints, showing true versus false points

# Use this flag to draw only neutrino pixels (broken right now)
NU_ONLY=False

# Often we have lots of spacepoints, to keep the visualization managable, 
#  we subsample the spacepoints if the following is True
CAP_PLOTTED_POINTS = True 
MAX_POINTS = 200000

xyz     = data['spacepoint_t']
truth   = data['truetriplet_t']
print("before cuts: ")
print("xyz: ",xyz.shape)
print("truth: ",truth.shape)

if NU_ONLY:
    origin  = data['origin_t']
    xyz = xyz[origin==1]
    truth = truth[origin==1]
    print("post nu-only")
    print("xyz: ",xyz.shape)
    print("truth: ",truth.shape)
    
if CAP_PLOTTED_POINTS and xyz.shape[0]>MAX_POINTS:
    reduction_factor = 150e3/float(xyz.shape[0])
    keep = np.random.rand( xyz.shape[0] ) < reduction_factor
    xyz   = xyz[keep==1]
    truth = truth[keep==1]
    #trackid = trackid[keep==1]
    
print("after cuts:")
print("xyz: ",xyz.shape)
print("truth: ",truth.shape)

plot = {
    "type":"scatter3d",
    "x": xyz[:,0],
    "y": xyz[:,1],
    "z": xyz[:,2],
    "mode":"markers",
    "name":"spacepoints",
    "marker":{"color":truth,"size":1,"opacity":0.5,"colorscale":"viridis"},
}

plot_list = [tpclines, plot]

#PLOT VTX
if kploader.has_larbysmc:
    vtx = np.zeros((1,3))
    vtx[0,0] = kploader.vtx_sce_x
    vtx[0,1] = kploader.vtx_sce_y
    vtx[0,2] = kploader.vtx_sce_z
    print("VTX: ",vtx)
    plot_vtx = {
        "type":"scatter3d",
            "x": vtx[:,0],
            "y": vtx[:,1],
            "z": vtx[:,2],
            "mode":"markers",
            "name":"VTX",
            "marker":{"color":"rgb(120,120,120)","size":10,"opacity":0.5,"colorscale":'viridis'},
        }
    plot_list += [plot_vtx]

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='Spacepoints with True labels',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=plot_list, layout=layout)
fig.show()

In [ ]:
# Plot Particle Class Labels

# Use this flag to draw only neutrino pixels (broken right now)
NU_ONLY=False

# Often we have lots of spacepoints, to keep the visualization managable, 
#  we subsample the spacepoints if the following is True
CAP_PLOTTED_POINTS = True 
MAX_POINTS = 200000

xyz     = data['spacepoint_t']
truth   = data['truetriplet_t']
print("before cuts: ")
print("xyz: ",xyz.shape)
print("truth: ",truth.shape)

if NU_ONLY:
    origin  = data['origin_t']
    xyz = xyz[origin==1]
    truth = truth[origin==1]
    print("post nu-only")
    print("xyz: ",xyz.shape)
    print("truth: ",truth.shape)
    
if CAP_PLOTTED_POINTS and xyz.shape[0]>MAX_POINTS:
    reduction_factor = 150e3/float(xyz.shape[0])
    keep = np.random.rand( xyz.shape[0] ) < reduction_factor
    xyz   = xyz[keep==1]
    truth = truth[keep==1]
    #trackid = trackid[keep==1]
    
print("after cuts:")
print("xyz: ",xyz.shape)
print("truth: ",truth.shape)

plot = {
    "type":"scatter3d",
    "x": xyz[:,0],
    "y": xyz[:,1],
    "z": xyz[:,2],
    "mode":"markers",
    "name":"span<%d"%(MATCH_SPAN),
    "marker":{"color":truth,"size":1,"opacity":0.5,"colorscale":"viridis"},
}

plot_list = [tpclines, plot]

#PLOT VTX
if kploader.has_larbysmc:
    vtx = np.zeros((1,3))
    vtx[0,0] = kploader.vtx_sce_x
    vtx[0,1] = kploader.vtx_sce_y
    vtx[0,2] = kploader.vtx_sce_z
    print("VTX: ",vtx)
    plot_vtx = {
        "type":"scatter3d",
            "x": vtx[:,0],
            "y": vtx[:,1],
            "z": vtx[:,2],
            "mode":"markers",
            "name":"VTX",
            "marker":{"color":"rgb(120,120,120)","size":10,"opacity":0.5,"colorscale":'viridis'},
        }
    plot_list += [plot_vtx]

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='Spacepoints with True labels',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=plot_list, layout=layout)
fig.show()

In [ ]:
# Plot instance labels

In [ ]:
# Plot ancestor labels

In [ ]:
# Plot keypoint score labels

# Voxelizer

Some of the neural networks we use work on voxel data. So we need a method to define a 3D array inside the detector and then fill the spacepoints into that array.

The class that converts spacepoint (or triplet) information is the `larflow::voxelizer::VoxelizeTriplet` class.  

For more info on the class see [VoxelizeTriplet.h](https://github.com/NuTufts/larflow/blob/master/larflow/Voxelizer/VoxelizeTriplets.h)

In [ ]:
# The Voxelizer Class
# See larflow/larflow/Voxelizer/VoxelizeTriplets.h for more info
# 
voxelizer = larflow.voxelizer.VoxelizeTriplets()


In [ ]:
voxeldata = voxelizer.get_full_voxel_labelset_dict( kploader )
# this returns a python list of dictionaries. 
# the latter contains voxel data and labels for each TPC in the detector.
# (for detectors with more than one TPC. MicroBooNE has just one.)

In [ ]:
# pick out one TPC's worth of data
vdata = voxeldata[0]
print(vdata.keys())
print("voxel coordinate tensor: ",vdata['voxcoord'].shape)
print("voxel feature tensor: ",vdata['voxfeat'].shape)
print("voxel ssnet  label tensor: ",vdata['ssnet_labels'].shape)
print("voxel true/ghost labels: ",vdata['voxlabel'].shape)
print("voxel array origin: ",vdata['voxorigin'].shape)

print("unique ssnet labels in tensor: ",np.unique(vdata['ssnet_labels']))

## Visualize the voxel data and labels

In [ ]:
# Plot Voxels

# we plot in (x,y,z) though what we've stored is indices in the 3D voxel array
# so we must convert
voxel_width = voxelizer.get_voxel_size()
origin = voxelizer.get_origin()

# scale voxel index to cm
xyz = vdata['voxcoord']*voxel_width
xyz[:,0] += origin[0]
xyz[:,1] += origin[1]
xyz[:,2] += origin[2]

# get spacepoint labels
truth = vdata['voxlabel']

plot = {
    "type":"scatter3d",
    "x": xyz[:,0],
    "y": xyz[:,1],
    "z": xyz[:,2],
    "mode":"markers",
    "name":"spacepoints",
    "marker":{"color":truth,"size":1,"opacity":0.5,"colorscale":"viridis"},
}

plot_list = [tpclines, plot]

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}

layout = go.Layout(
    title='Voxels colored by True labels',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=plot_list, layout=layout)
fig.show()

## Save the voxel to file

There are few options to save the data. 

* you can save the numpy arrays using numpy's native io methods (savez_compressed)
* you can save the data in a `larcv::SparseTensor2D` format, which is used in [lartpc_mlreco2d](https://github.com/NuTufts/lartpc_mlreco3d)
* you can save the data in `larcv::NumpyArrayFloat` or `larcv::NumpyArrayInt` which is class that was created to save the data in a form close to numpy arrays while also stored in ROOT trees

### saving to larcv::SparseTensor2D format

In [ ]:
# first make a VoxelSet, this is a container that holds the description of the active voxels 
from larcv import larcv

# translation from how we label particle types to how lartpc_mlreco3d labeled particles
# mlreco particle labels found in lartpc_mlreco3d/mlreco/utils/groups.py
# larflow particle labels found in larflow/larflow/PrepFlowMatchData/PrepSSNetTriplet.h
larcv2mlreco = {0:5, # bg -> ghost
                1:1, # electron -> electron
                2:0, # photon -> photon
                3:2, # muon -> muon 
                4:3, # pion -> pion
                5:4, # proton -> proton
                6:3, # other (usually mesons) -> pion
               }

# get the voxel arrays we need
voxcoord = vdata['voxcoord']
voxfeat  = vdata['voxfeat']
voxssnet = vdata['ssnet_labels']
print()

# each voxel needs an ID number. We use the unrolled array position
# to calculate this, we need array strides for each dimension
strides = [ voxelizer.get_nvoxels()[0]*voxelizer.get_nvoxels()[1],
            voxelizer.get_nvoxels()[2],
            1]

# create an output file
out_larcv = larcv.IOManager( larcv.IOManager.kWRITE, "voxeldata" )
out_larcv.set_out_file( "out_tutorial_voxels.root" )
out_larcv.initialize()


vset_uplane = larcv.VoxelSet() # save y-plane charge
vset_vplane = larcv.VoxelSet() # save y-plane charge
vset_yplane = larcv.VoxelSet() # save y-plane charge
vset_ssnet  = larcv.VoxelSet() # save particle labels

# some voxelset functions work best when voxels are added in sorted order (by ID number)
# so we do that first
idnums = []
id2index = {}
for n in range( voxcoord.shape[0] ):
    # create a voxel: this consists of an ID and value.
    # the ID is the array index
    idnum = int( voxcoord[n,0]*strides[0] + voxcoord[n,1]*strides[1] + voxcoord[n,2]*strides[2] )
    idnums.append(idnum)
    id2index[idnum] = n
    
# sort the different idnumbers
idnums.sort()
for idnum in idnums:
    # this format only allows us one feature: here filling with Y-plane values 
    vset_uplane.add( larcv.Voxel( idnum, voxfeat[id2index[idnum],0] ) )
    vset_vplane.add( larcv.Voxel( idnum, voxfeat[id2index[idnum],1] ) )
    vset_yplane.add( larcv.Voxel( idnum, voxfeat[id2index[idnum],2] ) )
    ssnetlabel = int(voxssnet[id2index[idnum]])
    # there needs to be come translation between our label definitions to lartpc_mlreco3d label definitions
    
    mlrecolabel = larcv2mlreco[ssnetlabel]
    vset_ssnet.add( larcv.Voxel(idnum, float(mlrecolabel) ))

# now we need the meta, that helps us go from array index to position in detector space
meta3d = larcv.Voxel3DMeta()
"""
inline void set(double xmin, double ymin, double zmin,
                    double xmax, double ymax, double zmax,
                    size_t xnum,size_t ynum,size_t znum,
                    DistanceUnit_t unit=kUnitCM)
"""
meta3d.set( voxelizer.get_origin()[0], voxelizer.get_origin()[1], voxelizer.get_origin()[2],
            voxelizer.get_origin()[0]+voxelizer.get_dim_len()[0],
            voxelizer.get_origin()[1]+voxelizer.get_dim_len()[1],
            voxelizer.get_origin()[2]+voxelizer.get_dim_len()[2],
            voxelizer.get_nvoxels()[0], voxelizer.get_nvoxels()[1], voxelizer.get_nvoxels()[2] )


# we need an event container for the tensor. 
# we get one from the IOManager so that is all setup to be saved to the root tree
# note the name of the containers is based on the lartpc_mlreco3d config 'config_uresnet_ppn.cfg'
#  * for charge data: "pcluster"
#  * for ssnet labels: "pcluster_semantic"
event_vox3d_uplane = out_larcv.get_data( larcv.kProductSparseTensor3D, "pcluster_uplane" )
event_vox3d_vplane = out_larcv.get_data( larcv.kProductSparseTensor3D, "pcluster_vplane" )
event_vox3d_yplane = out_larcv.get_data( larcv.kProductSparseTensor3D, "pcluster_yplane" )
event_vox3d_ssnet  = out_larcv.get_data( larcv.kProductSparseTensor3D, "pcluster_semantic" )
# add the data
event_vox3d_uplane.set( vset_uplane, meta3d )
event_vox3d_vplane.set( vset_vplane, meta3d )
event_vox3d_yplane.set( vset_yplane, meta3d )
event_vox3d_ssnet.set(  vset_ssnet,  meta3d )

# save the event to file
out_larcv.set_id(  0, 0, 0) # set an arbitrary (run,subrun,event) index
out_larcv.save_entry()

# finalize the file (properly close file)
out_larcv.finalize()

